In [1]:
import os
import numpy as np
import torch

device = 'cpu'
block_size = 399
batch_size = 2
dataset = ''

# poor man's data loader
data_dir = dataset
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint8, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint8, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # Ensure the starting index is a multiple of block_size
    ix = torch.randint(0, len(data) // (block_size + 1), (batch_size,)) * (block_size + 1)
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [2]:
print(len(train_data))

8951600
88.671875


In [3]:
import pickle
import os


# Usage
file_path = 'meta.pkl'
with open(file_path, 'rb') as f:
    data = pickle.load(f)
print(data)

itos = data['itos']

def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

{'vocab_size': 17, 'itos': {0: '\n', 1: ' ', 2: '-', 3: '.', 4: '/', 5: '0', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: ';', 16: 'x'}, 'stoi': {'\n': 0, ' ': 1, '-': 2, '.': 3, '/': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, ';': 15, 'x': 16}}


In [4]:
X, Y = get_batch('val')
print("Batch")
print(X)
print("y")
print(Y)

Batch
tensor([[15,  6,  3,  1,  6,  6,  2,  6, 10,  1,  7,  7,  2,  6, 12,  1,  7,  3,
          1,  6, 10,  2,  6, 14,  1,  7,  9, 16,  6, 10,  1,  8,  3,  1,  6,  5,
         16,  6, 14,  1,  7,  8, 16,  6, 11,  1,  9,  3,  1,  6,  7, 16,  6, 14,
          1,  7, 10,  2,  7,  7,  1, 10,  3,  1, 13,  2,  6,  6,  1,  8,  5,  2,
          7, 10,  1, 11,  3,  1,  9,  2, 13,  1,  7,  7,  2,  6, 13,  1, 12,  3,
          1, 14,  2,  6,  8,  1,  6, 13,  2,  6,  9,  1, 13,  3,  1,  6,  8, 16,
          7,  7,  1,  7, 10, 16,  6, 13,  1, 14,  3,  1,  6,  6,  2,  6, 11,  1,
          7, 12,  2,  7,  9,  1,  6,  5,  3,  1, 13,  2,  6,  7,  1,  7,  9, 16,
          6, 10,  1,  6,  6,  3,  1, 12,  2,  6,  5,  1,  6,  9, 16, 12,  1,  6,
          7,  3,  1,  8, 16,  6,  5, 16,  6, 14,  1,  7, 11,  2,  7,  7,  1,  6,
          8,  3,  1,  6, 14,  2,  7,  9,  1,  7, 13, 16,  6, 14,  1,  6,  9,  3,
          1,  6, 11, 16,  7,  8,  1,  6, 13,  2,  6, 10,  1,  6, 10,  3,  1,  7,
          2, 12,  1,  

In [5]:
print(decode(X[0].tolist()))
print(decode(X[1].tolist()))
print(decode(Y[0].tolist()))
print(decode(Y[1].tolist()))

;1. 11-15 22-17 2. 15-19 24x15 3. 10x19 23x16 4. 12x19 25-22 5. 8-11 30-25 6. 4-8 22-18 7. 9-13 18-14 8. 13x22 25x18 9. 11-16 27-24 10. 8-12 24x15 11. 7-10 14x7 12. 3x10x19 26-22 13. 19-24 28x19 14. 16x23 18-15 15. 2-7 22-18 16. 5-9 29-25 17. 9-13 32-28 18. 6-9 28-24 19. 7-10 15x6 20. 1x10 24-19 21. 10-14 18-15 22. 14-18 15-10 23. 9-14 10-6 24. 13-17 6-1 25. 17-22 1-6 26. 22x29 6-10 27. 29-25 10x
;1. 11-15 23-18 2. 8-11 27-23 3. 4-8 23-19 4. 10-14 19x10 5. 14x23 26x19 6. 7x14 24-20 7. 6-10 22-17 8. 9-13 30-26 9. 13x22 25x18x9 10. 5x14 26-23 11. 11-15 29-25 12. 15x24 28x19 13. 8-11 31-26 14. 11-15 32-28 15. 15x24 28x19 16. 3-7 25-22 17. 7-11 22-18 18. 1-5 18x9 19. 5x14 19-16 20. 12x19 23x16x7 21. 2x11 26-22 22. 10-15 22-17 23. 14-18 1/2-1/2                                                  
1. 11-15 22-17 2. 15-19 24x15 3. 10x19 23x16 4. 12x19 25-22 5. 8-11 30-25 6. 4-8 22-18 7. 9-13 18-14 8. 13x22 25x18 9. 11-16 27-24 10. 8-12 24x15 11. 7-10 14x7 12. 3x10x19 26-22 13. 19-24 28x19 14. 16